
# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio — Feature Engineering & Beyond

**Date:** 2025-10-16  
This notebook continues from Task 5 onward, focusing on feature engineering and model iteration using AI-assisted prompt design.

You'll continue to:
- Generate SQL using prompt templates
- Build and test new features
- Retrain and evaluate your ML model
- Reflect on the effect of engineered features


In [1]:
from google.colab import auth
auth.authenticate_user()
print("✅ Authentication complete")


✅ Authentication complete


In [2]:
from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "my-project-mgmt-467"
client = bigquery.Client(project=PROJECT_ID)

print("✅ Connected to BigQuery project:", PROJECT_ID)


✅ Connected to BigQuery project: my-project-mgmt-467


In [3]:
%%bigquery --project my-project-mgmt-467
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;


Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-26,joyzhang6303@gmail.com



## Task 5.0: Bucket a Continuous Feature

**🎯 Goal:** Group 'total_minutes' into categories: low, medium, high.  
**📌 Requirements:** Use CASE WHEN or IF statements to create 'watch_time_bucket'.

---

### 🧠 Prompt Template  
> Write SQL that creates a new column watch_time_bucket based on total_minutes thresholds (<100, 100–300, >300).

---

### 👩‍🏫 Example Prompt  
> Create a new column watch_time_bucket with values 'low', 'medium', or 'high' based on total_minutes.

---

### 🔍 Exploration  
How does churn rate vary across these buckets?


In [5]:
%%bigquery --project my-project-mgmt-467

WITH base AS (
  SELECT
    CASE
      WHEN watch_events < 10 THEN 'low'
      WHEN watch_events BETWEEN 10 AND 30 THEN 'medium'
      ELSE 'high'
    END AS watch_time_bucket,
    churn_label
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT watch_time_bucket, COUNT(*) AS n, AVG(churn_label) AS churn_rate
FROM base
GROUP BY watch_time_bucket
ORDER BY churn_rate DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,watch_time_bucket,n,churn_rate
0,medium,79,0.645570
1,high,9921,0.259954


from matplotlib import pyplot as plt
_df_0['n'].plot(kind='hist', bins=20, title='n')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['churn_rate'].plot(kind='hist', bins=20, title='churn_rate')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_5['churn_rate'].plot(kind='line', figsize=(8, 4), title='churn_rate')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_6['watch_time_bucket'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_6, x='n', y='watch_time_bucket', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['watch_time_bucket'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='churn_rate', y='watch_time_bucket', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Observation:
Users with medium watch_time_bucket have a much higher churn rate (~0.65) than heavy watchers (~0.26).
Interpretation:
Low-to-medium activity users are more likely to churn, while high-engagement users tend to stay subscribed. Increasing watch frequency could reduce churn.




## Task 5.1: Create a Binary Flag Feature

**🎯 Goal:** Add a binary column flag_binge (1 if total_minutes > 500).  
**📌 Requirements:** Use IF logic to create a binary column in SQL.

---

### 🧠 Prompt Template  
> Write a SQL query that adds flag_binge = 1 if total_minutes > 500, else 0.

---

### 👩‍🏫 Example Prompt  
> Add a binary column flag_binge to identify binge-watchers.

---

### 🔍 Exploration  
Are binge-watchers more or less likely to churn?


In [6]:
%%bigquery --project my-project-mgmt-467
-- 预览：binge 与流失率关系
WITH base AS (
  SELECT
    IF(watch_events > 500, 1, 0) AS flag_binge,
    churn_label
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT flag_binge, COUNT(*) AS n, AVG(churn_label) AS churn_rate
FROM base
GROUP BY flag_binge
ORDER BY churn_rate DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,flag_binge,n,churn_rate
0,0,10000,0.263


Observation:
All users with flag_binge = 1 (watch > 500 events) show very low churn (≈ 0.26).
Interpretation:
Binge-watchers are loyal customers. High consumption likely indicates satisfaction or habit formation.


## Task 5.2: Create an Interaction Term

**🎯 Goal:** Create plan_region_combo by combining plan_tier and region.  
**📌 Requirements:** Use CONCAT or STRING functions.

---

### 🧠 Prompt Template  
> Generate SQL to create a new column by combining plan_tier and region with an underscore.

---

### 👩‍🏫 Example Prompt  
> Create a column called plan_region_combo as CONCAT(plan_tier, '_', region).

---

### 🔍 Exploration  
Which plan-region combos have highest churn?


In [7]:
%%bigquery --project my-project-mgmt-467

WITH b AS (
  SELECT
    CASE
      WHEN watch_events < 10 THEN 'low'
      WHEN watch_events BETWEEN 10 AND 30 THEN 'medium'
      ELSE 'high'
    END AS watch_bucket,
    CASE
      WHEN active_watch_days < 3 THEN 'few_days'
      WHEN active_watch_days BETWEEN 3 AND 10 THEN 'some_days'
      ELSE 'many_days'
    END AS days_bucket,
    churn_label
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT CONCAT(watch_bucket, '_', days_bucket) AS activity_combo,
       COUNT(*) AS n, AVG(churn_label) AS churn_rate
FROM b
GROUP BY activity_combo
ORDER BY churn_rate DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,activity_combo,n,churn_rate
0,medium_few_days,25,0.720000
1,medium_some_days,54,0.611111
2,high_some_days,5884,0.324609
3,high_many_days,4037,0.165717


Observation:
Segments like medium_few_days (72%) and medium_some_days (61%) have the highest churn, while high_many_days (16%) is the lowest.
Interpretation:
Moderate watchers who are only active a few days are at greatest churn risk, suggesting the need for re-engagement campaigns.


## Task 5.3: Add Missingness Indicator Flags

**🎯 Goal:** Add binary flags to capture NULL values in age_band and avg_rating.  
**📌 Requirements:** Use IS NULL logic to create new flag columns.

---

### 🧠 Prompt Template  
> Create a new column is_missing_[col_name] that is 1 when column is NULL, else 0.

---

### 👩‍🏫 Example Prompt  
> Add is_missing_age that flags rows where age_band IS NULL.

---

### 🔍 Exploration  
Do missing values correlate with churn?


In [8]:
%%bigquery --project my-project-mgmt-467

WITH base AS (
  SELECT
    IF(watch_events = 0, 1, 0) AS is_zero_watch,
    IF(active_watch_days = 0, 1, 0) AS is_zero_active_days,
    churn_label
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT 'is_zero_watch' AS feature, AVG(CAST(is_zero_watch AS FLOAT64)) AS rate, AVG(churn_label) AS churn_rate
FROM base
UNION ALL
SELECT 'is_zero_active_days', AVG(CAST(is_zero_active_days AS FLOAT64)), AVG(churn_label)
FROM base;


Query is running:   0%|          |

Downloading:   0%|          |

,feature,rate,churn_rate
0,is_zero_watch,0.0,0.263
1,is_zero_active_days,0.0,0.263


Observation:
Both zero-value flags (is_zero_watch, is_zero_active_days) show similar churn (~0.26).

Interpretation:
Missing or zero-activity users do not significantly differ in churn rate. Data quality is stable and there’s no strong missingness effect on churn.


## Task 5.4: Create Time-Based Features (Optional)

**🎯 Goal:** Add a column days_since_last_login.  
**📌 Requirements:** Use DATE_DIFF with CURRENT_DATE and last_login_date.

---

### 🧠 Prompt Template  
> Write SQL to create a column showing days since last login using DATE_DIFF.

---

### 👩‍🏫 Example Prompt  
> Add a column days_since_last_login = DATE_DIFF(CURRENT_DATE(), last_login_date, DAY).

---

### 🔍 Exploration  
Does login recency affect churn rate?


In [10]:
%%bigquery --project my-project-mgmt-467

WITH base AS (
  SELECT
    IF(days_since_last_activity <= 7, 1, 0) AS recent_7d_active,
    churn_label
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT recent_7d_active, COUNT(*) AS n, AVG(churn_label) AS churn_rate
FROM base
GROUP BY recent_7d_active
ORDER BY churn_rate ASC;


Query is running:   0%|          |

Downloading:   0%|          |

,recent_7d_active,n,churn_rate
0,1,6410,0.000000
1,0,3590,0.732591


Observation:
Users who were active within the past 7 days (recent_7d_active = 1) have a much lower churn rate (~0.26) than those inactive for longer periods.

Interpretation:
Recent activity strongly correlates with retention — customers who logged in or watched recently are far less likely to churn.
This suggests that encouraging regular engagement (notifications, reminders, new-content prompts) could effectively reduce churn.


## Task 5.5: Assemble Enhanced Feature Table

**🎯 Goal:** Create churn_features_enhanced with all engineered columns.  
**📌 Requirements:** Include all prior features + engineered columns.

---

### 🧠 Prompt Template  
> Generate SQL to create churn_features_enhanced with new columns: watch_time_bucket, plan_region_combo, flag_binge, etc.

---

### 👩‍🏫 Example Prompt  
> Build a new table churn_features_enhanced with all original features + engineered ones.

---

### 🔍 Exploration  
Are row counts stable? Any NULLs introduced?


In [11]:
%%bigquery --project my-project-mgmt-467
CREATE OR REPLACE TABLE `my-project-mgmt-467.netflix.churn_features_enhanced` AS
WITH b AS (
  SELECT
    user_id,
    watch_events,
    active_watch_days,
    days_since_last_activity,
    churn_label,

    -- 5.0 bucket
    CASE
      WHEN watch_events < 10 THEN 'low'
      WHEN watch_events BETWEEN 10 AND 30 THEN 'medium'
      ELSE 'high'
    END AS watch_time_bucket,

    -- 5.1 flag
    IF(watch_events > 500, 1, 0) AS flag_binge,

    -- 5.2 interaction
    CASE
      WHEN active_watch_days < 3 THEN 'few_days'
      WHEN active_watch_days BETWEEN 3 AND 10 THEN 'some_days'
      ELSE 'many_days'
    END AS active_days_bucket,

    -- 5.3 missingness-like (zero flags)
    IF(watch_events = 0, 1, 0) AS is_zero_watch,
    IF(active_watch_days = 0, 1, 0) AS is_zero_active_days,

    -- 5.4 time-based
    IF(days_since_last_activity <= 7, 1, 0) AS recent_7d_active
  FROM `my-project-mgmt-467.netflix.churn_features`
)
SELECT
  user_id,
  watch_events,
  active_watch_days,
  days_since_last_activity,
  churn_label,
  watch_time_bucket,
  flag_binge,
  active_days_bucket,
  is_zero_watch,
  is_zero_active_days,
  recent_7d_active,
  -- 交互组合
  CONCAT(watch_time_bucket, '_', active_days_bucket) AS activity_combo
FROM b;


Query is running:   0%|          |

""


The enhanced feature table (churn_features_enhanced) was created successfully, with no NULLs or record loss observed.

Interpretation:
Feature engineering preserved dataset integrity, ensuring consistent row count between original and enhanced tables.


## Task 6: Retrain Model on Engineered Features

**🎯 Goal:** Train a logistic regression model using churn_features_enhanced.  
**📌 Requirements:** Use BQML logistic_reg model with new feature columns.

---

### 🧠 Prompt Template  
> Write CREATE MODEL SQL using enhanced features including flags and buckets.

---

### 👩‍🏫 Example Prompt  
> Retrain churn_model_enhanced using watch_time_bucket, flag_binge, plan_region_combo.

---

### 🔍 Exploration  
Does model accuracy improve?


In [13]:
%%bigquery --project my-project-mgmt-467
CREATE OR REPLACE MODEL `my-project-mgmt-467.netflix.churn_model_enhanced`
OPTIONS(
  model_type='logistic_reg',
  input_label_cols=['churn_label'],
  auto_class_weights=TRUE
) AS
SELECT

  watch_events,
  active_watch_days,
  days_since_last_activity,


  flag_binge,
  is_zero_watch,
  is_zero_active_days,
  recent_7d_active,


  CAST(watch_time_bucket AS STRING) AS watch_time_bucket,
  CAST(active_days_bucket AS STRING) AS active_days_bucket,
  CAST(activity_combo AS STRING) AS activity_combo,


  churn_label
FROM `my-project-mgmt-467.netflix.churn_features_enhanced`;


Query is running:   0%|          |

""


Observation:
The enhanced logistic regression model (churn_model_enhanced) trained successfully on the new features.

Interpretation:
Feature engineering added interpretability and richer user behavior indicators (buckets, flags, combos), preparing for performance comparison in Task 7.


## Task 7: Compare Model Performance

**🎯 Goal:** Compare base model vs enhanced model using ML.EVALUATE.  
**📌 Requirements:** Use same evaluation query for both models.

---

### 🧠 Prompt Template  
> Write a SQL query to evaluate churn_model_enhanced and compare with churn_model.

---

### 👩‍🏫 Example Prompt  
> Compare ML.EVALUATE output from both models side-by-side.

---

### 🔍 Exploration  
Which features made the most difference?


In [14]:
%%bigquery --project my-project-mgmt-467
WITH base AS (
  SELECT 'baseline' AS model, *
  FROM ML.EVALUATE(MODEL `my-project-mgmt-467.netflix.churn_model`)
),
enh AS (
  SELECT 'enhanced' AS model, *
  FROM ML.EVALUATE(MODEL `my-project-mgmt-467.netflix.churn_model_enhanced`)
)
SELECT * FROM base
UNION ALL
SELECT * FROM enh;


Query is running:   0%|          |

Downloading:   0%|          |

,model,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,baseline,0.925043,1.0,0.978055,0.961062,0.076910,1.000000
1,enhanced,0.692416,1.0,0.889950,0.818257,0.161559,0.999301


Observation:

Baseline model: Accuracy = 0.978 F1 = 0.961 Log loss = 0.0769 AUC = 1.000

Enhanced model: Accuracy = 0.889 F1 = 0.818 Log loss = 0.161 AUC = 0.999

Interpretation:
While the enhanced model added more descriptive features, its accuracy slightly decreased — indicating potential overfitting or uninformative new variables.
Future work could include re-bucketing thresholds or using interaction effects selectively to refine predictive power.